<a href="https://colab.research.google.com/github/thazin31086/WebTrace/blob/master/UniversalSentenceEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)


data = pd.read_csv('SVF.csv')

y = list(data['label']) 
x = list(data['Title']) 

le = preprocessing.LabelEncoder()
le.fit(y)


def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

test = encode(le, ['question','question|help wanted', 'question|enhancement', 'question|enhancement|help wanted'])

untest = decode(le, test)


x_enc = x
y_enc = encode(le, y)

x_train = np.asarray(x_enc[:40])
y_train = np.asarray(y_enc[:40])

x_test = np.asarray(x_enc[10:])
y_test = np.asarray(y_enc[10:])

from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))


input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(4, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=3, batch_size=32)
    model.save_weights('./model.h5')
    
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./model.h5')  
    predicts = model.predict(x_test, batch_size=32)

    
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

metrics.confusion_matrix(y_test, y_preds)

print(metrics.classification_report(y_test, y_preds))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Epoch 1/3
40/40 [==============================] - 19s 478ms/step - loss: 1.3680 - acc: 0.3750
Epoch 2/3
40/40 [==============================] - 0s 12ms/step - loss: 1.2055 - acc: 0.8250
Epoch 3/3
40/40 [==============================] - 0s 11ms/step - loss: 1.0696 - acc: 0.8500
                                  precision    recall  f1-score   support

                        question       0.88      1.00      0.93        35
            question|enhancement       0.00      0.00      0.00         3
question|enhancement|help wanted       0.00      0.00      0.00         1
            question|help wanted       0.00      0.00      0.00         1

                        accuracy                           0.88        40
                       macro avg       0.22      0.25      0.23        40
                    weighted avg       0.77      0.88      0.82        40



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# New Section